# AI MEGADADOS 23-1

## Parte 1 - CarTracking!

Na parte 1 da prova de MEGADADOS, iremos trabalhar com a base de dados sintética **CarTracking**.

### Insper autograding!

Para receber feedback dos exercício na parte 1, iremos utilizar o `insper autograding`. Se não fez este passo, abra o notebook disponível na aula 09.

**Sugestão**: para aproveitar o `.env`, crie uma pasta para a prova no mesmo local onde costuma deixar as pastas das **aulas**!

### Instalação da base

Execute os scripts `script_001.sql` e `script_002.sql` no MySQL Workbench. Estes scripts criam uma base **cartracking** e inserem alguns dados de exemplo para resolução da prova.

A base pode ser representada pelo seguinte diagrama do model orelacional (diagrama ER):

<img src="img/deer_cartracing.png">

A base da prova simula um cenário de **rastreamento de veículos**. Nela, estão cadastrados **automóveis** de **clientes**. Os automóveis podem ter múltiplos **rastreadores** instalados, mas cada rastreador está associado a apenas um automóvel.

Os rastreadores irão gerar **eventos** de rastreamento, contendo informações de geolocalização e temperatura em determinados momentos do tempo.

## Como resolver os exercícios?

Crie a base da prova em sua máquina (passo anterior). Utilize o MySQL Workbench ou o conector para testar as queries. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [ ]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [ ]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
        database='cartracking',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

### Notas

Para conferir a nota da correção automática da prova, utilize:

In [ ]:
ia.grades(task='ai_md_23_1')

In [ ]:
ia.grades(by='TASK', task='ai_md_23_1')

**Obs**: Esta parte da avaliação vale **6 pontos** na nota da prova.
- 50% da nota será pela correção automática do servidor
- 50% da nota será pela correção pelo professor, que levará em consideração, por exemplo:
    - Quão distante sua solução ficou de uma solução correta
    - Se as queries estão formatadas:
        - Padrão de maiúsculas e minúsculas
        - Tabulações e quebras de linhas
    - Se utiliza *alias* nas tabelas, especialmente em queries com múltiplas tabelas
    
A nota retornada pelo servidor estará no intervalo `0.0` a `10.0` e será multiplicada por `0.3` para compor os `50%` da correção automática.

Ainda, considere que os testes não são exaustivos. Sua solução deve funcionar para qualquer conjunto de dados no *schema* **cartracking**.

**Exercício 1**: Crie uma query que retorne a quantidade de clientes pessoa jurídica cadastrados na base.

**Obs**: considere que um cliente pessoa jurídica é um cliente com cnpj preenchido!

In [ ]:
sql_ex01 = '''
SELECT * FROM cliente
WHERE cnpj IS NOT NULL
'''

db(sql_ex01)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex01', task='ai_md_23_1', question='ex01', answer_type='pyvar')

**Exercício 2**: Crie uma query que retorne, **sem repetição**, todos os `modelo` e `marca` dos `automovel` cadastrados. Retorne ordenado de forma crescente pelo `modelo`.

In [ ]:
sql_ex02 = '''
SELECT DISTINCT modelo.modelo, marca.marca
FROM automovel 
INNER JOIN modelo ON automovel.idmodelo = modelo.idmodelo
INNER JOIN marca ON modelo.idmarca = marca.idmarca
ORDER BY modelo ASC
'''

db(sql_ex02)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex02', task='ai_md_23_1', question='ex02', answer_type='pyvar')

**Exercício 3**: Crie uma query que retorne, o `nome` do `cliente` e a **quantidade** de automóveis que o cliente possui cadastrado na base.


**Obs**:
- Os clientes com mais automóveis cadastrados devem aparecer primeiro.
- Clientes sem automóveis cadastrados não devem ser retornados

In [ ]:
sql_ex03 = '''
SELECT cliente.nome, COUNT(automovel.idautomovel) AS quantidade
FROM automovel INNER JOIN cliente ON automovel.idcliente = cliente.idcliente
GROUP BY automovel.idcliente
ORDER BY quantidade DESC
'''

db(sql_ex03)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex03', task='ai_md_23_1', question='ex03', answer_type='pyvar')

**Exercício 4**: Crie uma query que retorne, nesta ordem, o `idautomovel`, `ano`, `idcliente`, `idmodelo` e `travado` dos automóveis que **não possuem nenhum rastreador** vinculado a eles.


**Obs**: ordene de forma crescente pelo `idautomovel`.

In [ ]:
sql_ex04 = '''
SELECT idautomovel, ano, idcliente, idmodelo, travado
FROM automovel
WHERE idautomovel NOT IN (SELECT idautomovel FROM rastreador WHERE idautomovel IS NOT NULL)
ORDER BY idautomovel ASC
'''

db(sql_ex04)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex04', task='ai_md_23_1', question='ex04', answer_type='pyvar')

**Exercício 5**: Considerando apenas os rastreadores **ativos**, crie uma query que retorne o `nome` do cliente, o `modelo` do automóvel rastreado, a temperatura **máxima** e **média** do motor registradas nos eventos do automóvel.

**Obs**:
- Retorne ordenado pela temperatura máxima, de forma decrescente
- Retorne apenas os registros com temperatura máxima do motor acima de `85`
- A coluna de temperatura máxima deve se chamar `max_temp_motor`
- A coluna de temperatura média deve se chamar `avg_temp_motor`

In [ ]:
sql_ex05 = '''
SELECT cliente.nome, modelo.modelo, MAX(evento.temperatura_motor) AS max_temp_motor, AVG(evento.temperatura_motor) AS avg_temp_motor
FROM cliente INNER JOIN automovel ON cliente.idcliente = automovel.idcliente
INNER JOIN modelo ON automovel.idmodelo = modelo.idmodelo
INNER JOIN rastreador ON automovel.idautomovel = rastreador.idautomovel
INNER JOIN evento ON rastreador.idrastreador = evento.idrastreador
GROUP BY evento.idrastreador
HAVING max_temp_motor > 85
'''

db(sql_ex05)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex05', task='ai_md_23_1', question='ex05', answer_type='pyvar')

**Exercício 6**: Crie uma tabela temporária `tmp_cliente_eventos` contendo o `nome`, `cpf`, `cnpj` do cliente, além da quantidade de **eventos** registrados na tabela `evento` para cada cliente.

Requisitos:
- Retorne ordenado pelo `nome` de forma crescente
- A coluna de contagem deve se chamar `qtde_eventos`
- Clientes sem eventos devem ter o valor `0` em `qtde_eventos`
- Nos clientes com o `cpf` ou `cnpj` nulos, retorne uma string `'NAO_INFORMADO'`

In [ ]:
sql_ex06 = '''
DROP TABLE IF EXISTS tmp_cliente_eventos;

CREATE TEMPORARY TABLE tmp_cliente_eventos
SELECT cliente.nome, cliente.cpf, cliente.cnpj, COUNT(evento.idevento) AS qtde_eventos
FROM cliente LEFT OUTER JOIN automovel ON cliente.idcliente = automovel.idcliente
LEFT OUTER JOIN modelo ON automovel.idmodelo = modelo.idmodelo
LEFT OUTER JOIN rastreador ON automovel.idautomovel = rastreador.idautomovel
LEFT OUTER JOIN evento ON rastreador.idrastreador = evento.idrastreador
GROUP BY cliente.idcliente;

UPDATE tmp_cliente_eventos SET cpf = 'NAO_INFORMADO' WHERE cpf IS NULL;
UPDATE tmp_cliente_eventos SET cnpj = 'NAO_INFORMADO' WHERE cnpj IS NULL;
'''

db(sql_ex06)

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex06', task='ai_md_23_1', question='ex06', answer_type='pyvar')

### Conferindo as notas!

In [ ]:
ia.grades(task='ai_md_23_1')

## Parte 2 - Confraternizar!

Você foi procurado por um grupo de alunos do Insper para uma consultoria na área de dados! Eles reclamaram que precisam realizar diversas confraternizações entre os estudantes do Insper, sendo difícil gerir todos os detalhes relativos aos eventos.

Ainda, é desejo dos organizadores que toda a gestão seja digital, sem mais ingressos no papel!

Em conversas com o cliente, você levantou as seguintes informações sobre o domínio de negócios:
- As confraternizações são agendadas para acontecer em determinada data.
- As confraternizações sempre possuem um tipo (calourada, pizzada, cine-cultural, etc.). Então, uma confraternização sempre é uma nova realização (ou oferecimento) de um tipo de evento.
- Cada confraternização pode ser organizada por diversos alunos. É preciso manter um cadastro dos organizadores, indicando o cargo ocupado em cada festa (quando o aluno fizer parte da organização).
- Os ingressos apenas podem ser adquiridos por alunos. O aluno pode comprar quantos ingresso quiser (para ele e/ou convidados). Quando o aluno compra um ingresso de determinada festa, ele tem a opção de indicar o nome do convidado que irá utilizar o ingresso.

Utilize estas informações para propor soluções adequadas para os próximos exercícios.

**Obs**: Esta parte da avaliação vale **4 pontos** na nota da prova:
- 1.50 para o exercício 7
- 0.75 para o exercício 8
- 0.75 para o exercício 9
- 1.00 para o exercício 10

**Exercício 7**: Desenhe o diagrama do modelo relacional deste problema. Não esqueça de indicar claramente os tipos, chaves primárias, chaves estrangeiras, e a cardinalidade dos relacionamentos.

**Obs:** Salve a imagem do diagrama na pasta `img`. Edite na resposta o caminho para a imagem!

<div class="alert alert-success">
    
Edite na resposta o caminho para a imagem!
    
<img src="img/confraternizacao.png">

</div>

**Exercício 8**: Cole a **DDL** para criação da tabela de `organizadores`.

<div class="alert alert-success">

```mysql

CREATE TABLE organizadores (
    id_organizador INTEGER NOT NULL AUTO_INCREMENT,
    id_aluno INTEGER NOT NULL,
    id_confraternizacao INTEGER NOT NULL,
    PRIMARY KEY (id_organizador),
    FOREIGN KEY (id_aluno) REFERENCES alunos(id_aluno),
    FOREIGN KEY (id_confraternizacao) REFERENCES confraternizacoes(id_confraternizacao)
);

```

</div>

**Exercício 9**: Cole a **DML** para inserir um registro na tabela de `organizadores`. Utilize dados quaisquer!

**Obs**: a query precisa estar correta, mas não necessariamente irá rodar no notebook do professor (pois pode depender de chaves estrangeiras). Não é obrigatório (nem vale nota) que você gaste tempo preenchendo valores em outras tabelas para que a query de fato funcione, mas tudo bem caso queira fazer assim!

<div class="alert alert-success">

```mysql

INSERT INTO organizadores (id_organizador, id_aluno, id_confraternizacao) 
VALUES (1, 1, 1);

```

</div>

**Exercício 10**: Discorra sobre as diferenças entre os relacionamentos identificadores e não-identificadores.

Esta questão será corrigida considerando a seguinte subrica:
| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

No contexto do SQL e do design de banco de dados, relacionamentos de identificação e não identificação são duas maneiras de representar associações entre tabelas em um banco de dados relacional. Esses relacionamentos são essenciais para manter a integridade dos dados e estabelecer conexões entre diferentes conjuntos de dados. Vamos explorar as diferenças entre esses dois tipos de relacionamento:

Identificando Relacionamento:

Em um relacionamento de identificação, a chave primária da tabela filha inclui uma ou mais colunas que não apenas servem como chaves estrangeiras para vinculá-la à tabela pai, mas também identificam exclusivamente cada registro na tabela filha.
Isto significa que a chave primária da tabela filha é composta por uma combinação de colunas de chave estrangeira e possivelmente algumas colunas adicionais.
Os relacionamentos de identificação são normalmente usados em relacionamentos um-para-um ou um-para-muitos, onde a existência da tabela filho depende da tabela pai.
Quando o registro pai é excluído, o registro filho também é excluído porque a chave primária do filho faz parte da chave primária do pai e eles estão fortemente acoplados.
Relacionamento não identificável:

Em um relacionamento sem identificação, a chave primária da tabela filha consiste em seus próprios atributos (colunas) e não inclui nenhuma coluna de chave estrangeira da tabela pai.
Relacionamentos não identificadores são frequentemente usados em relacionamentos um-para-muitos ou muitos-para-muitos, onde a tabela filha pode existir independentemente da tabela pai.
A exclusão de um registro pai em um relacionamento sem identificação não exclui necessariamente os registros filho associados porque a chave primária do filho não está vinculada à chave primária do pai.
Aqui está um exemplo para ilustrar a diferença entre os dois:

Considere duas tabelas, “Autor” e “Livro”.

Em um relacionamento de identificação, a tabela “Autor” pode ter uma chave primária “AuthorID”, e a chave primária da tabela “Livro” incluiria “AuthorID” como chave estrangeira, junto com outros atributos como “BookID”. A exclusão de um autor também excluiria todos os seus livros, pois os registros secundários (livros) são identificados exclusivamente por "AuthorID" e "BookID".

Em um relacionamento sem identificação, a tabela "Autor" pode ter "AuthorID" como sua chave primária, e a chave primária da tabela "Livro" incluiria apenas "BookID" e não faria referência a "AuthorID". Nesse caso, excluir um autor não excluiria automaticamente seus livros porque os registros filhos (livros) possuem sua própria chave primária (apenas "BookID") e não estão diretamente vinculados ao autor.

</div>

## Entrega!

É hora de entregar. Faça um **zip** do notebook + imagens chamado `ai_megadados.zip` e submeta no Blackboard!

**Obs**:
- `zip`, não use `rar`!
- Não precisa entregar os scripts fornecidos pelo professor (pasta scripts)